In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *
from setting import *

In [ ]:
signal = pd.read_table(
    PATH_DICT["feature_x_sample.context_signal.tsv"], index_col=0
).fillna(0)

In [ ]:
k_nmfcc = ccal.nmf_consensus_cluster_with_multiple_k(
    signal,
    NMF_KS,
    n_job=MAX_N_JOB,
    plot_df=signal.size < PLOT_HEAT_MAP_MAX_SIZE,
    plot_w=signal.shape[0] < PLOT_CLUSTER_MAX_SIZE,
    plot_h=signal.shape[1] < PLOT_CLUSTER_MAX_SIZE,
    directory_path=PATH_DICT["nmf/"],
)

In [ ]:
w = pd.read_table(PATH_DICT["w.tsv"], index_col=0)

h = pd.read_table(PATH_DICT["h.tsv"], index_col=0)

w.columns.name = "P"

h.columns.name = SAMPLE_ALIAS

In [ ]:
for w_or_h, w_or_h_df in (("w", w), ("h", h)):

    if w_or_h == "w":

        normalization_axis = 1

        cluster_axis = 0

    elif w_or_h == "h":

        normalization_axis = 0

        cluster_axis = 1

    if w_or_h_df.shape[cluster_axis] < PLOT_CLUSTER_MAX_SIZE:

        ccal.plot_heat_map(
            w_or_h_df,
            normalization_axis=normalization_axis,
            normalization_method="-0-",
            cluster_axis=cluster_axis,
            title="NMF K={} {}".format(NMF_K, w_or_h.title()),
            xaxis_title=w_or_h_df.columns.name,
            yaxis_title=w_or_h_df.index.name,
        )

In [ ]:
for w_or_h, w_or_h_df in (("w", w.T), ("h", h)):

    ccal.make_comparison_panel(
        w_or_h_df,
        w_or_h_df,
        axis=1,
        title="NMF K={} {}".format(NMF_K, w_or_h.title()),
        name_0=w_or_h_df.index.name,
        name_1=w_or_h_df.index.name,
    )

In [ ]:
for element_x_p, alias, elements_to_peek in (
    (w, FEATURE_ALIAS, FEATURES_TO_PEEK),
    (h.T, SAMPLE_ALIAS, SAMPLES_TO_PEEK),
):

    if "negative" in CONTEXTS and "positive" in CONTEXTS:

        elements_to_peek = tuple(
            ("(-) {}".format(element_name) for element_name in elements_to_peek)
        ) + tuple(("(+) {}".format(element_name) for element_name in elements_to_peek))

    ccal.plot_bubble_map(
        element_x_p.reindex(index=elements_to_peek).dropna(how="all"),
        title="{} to Peek".format(alias),
        xaxis_title="P",
        yaxis_title=alias,
    )

In [ ]:
feature_x_sample = pd.read_table(FEATURE_X_SAMPLE_FILE_PATH, index_col=0)

for (
    w_or_h,
    target_x_element,
    element_x_element_signal,
    element_x_element_raw,
    directory_path,
) in (
    ("w", w.T, signal.T, feature_x_sample.T, PATH_DICT["w|match/"]),
    ("h", h, signal, feature_x_sample, PATH_DICT["h|match/"]),
):

    ccal.make_match_panels(
        target_x_element,
        {
            "signal": {"df": element_x_element_signal, "data_type": "continuous"},
            FEATURE_X_SAMPLE_VALUE_NAME: {
                "df": element_x_element_raw,
                "data_type": "continuous",
            },
        },
        n_job=MAX_N_JOB,
        n_required_for_match_function=0.64,
        extreme_feature_threshold=EXTREME_FEATURE_THRESHOLD,
        plot_std=PLOT_STD,
        directory_path=directory_path,
    )

In [ ]:
for w_or_h, w_or_h_df in (("w", w.T), ("h", h)):

    for p, values in w_or_h_df.iterrows():

        ccal.select_series_low_and_high_index(
            values,
            title="NMF K={} {} {} Signature".format(NMF_K, w_or_h.title(), p),
            file_path_prefix="{}/{}".format(
                PATH_DICT["{}|signature/".format(w_or_h)], p
            ),
        )